# [Advent of Code 2021](https://adventofcode.com/2021)

NB: Installer jupyter lab i venv, og start så med `venv/bin/jupyter lab`

In [1]:
import re
import numpy as np
import math
from collections import Counter
import itertools
import collections

#from itertools import tee
#from itertools import chain
#from functools import lru_cache

# --- Day 1: Sonar Sweep ---

In [2]:
#puzzle_input = 'data/test.txt'
puzzle_input = 'data/aoc2021_day1.txt'
with open(puzzle_input) as f:
    data = []
    for line in f:
        data.append(int(line))
data[0:10]

[195, 197, 201, 204, 203, 216, 213, 215, 216, 185]

In [3]:
# How many measurements are larger than the previous measurement?
p1 = sum([1 if data[x+1] > data[x] else 0 for x in range(len(data)-1)])
print('Part 1: ', p1)

Part 1:  1624


In [4]:
# three-measurement sliding window
sw = [data[x] + data[x+1] + data[x+2] for x in range(len(data)-2)]
sw[0:10]

[593, 602, 608, 623, 632, 644, 644, 616, 589, 563]

In [5]:
p2 = sum([1 if sw[x+1] > sw[x] else 0 for x in range(len(sw)-1)])
print('Part 2: ', p2)

Part 2:  1653


# --- Day 2: Dive! ---

```
forward 5
down 5
forward 8
up 3
down 8
forward 2
```

In [6]:
puzzle_input = 'data/aoc2021_day2.txt'
course = []
with open(puzzle_input) as f:
    for line in f:
        forward = up = down = 0
        m = re.search(r"([a-z]+) (\d+)$", line)
        if m.group(1) == 'forward':
            forward = int(m.group(2))
        elif m.group(1) == 'up':
            up = int(m.group(2))
        elif m.group(1) == 'down':
            down = int(m.group(2))
        direction = (forward, up, down)
        course.append(direction) # [(5, 0, 0), (0, 0, 5), ...]

sum_forward = sum(forward for forward, up, down in course)
sum_up = sum(up for forward, up, down in course)
sum_down = sum(down for forward, up, down in course)

p1 = sum_forward * (sum_down - sum_up)
print('Part 1: ', p1)

Part 1:  1990000


In [7]:
forward = depth = aim = 0
for x in course:
    if x[0]>0:
        forward += x[0]
        depth += aim * x[0]
    elif x[1]>0:
        aim -= x[1]
    elif x[2]>0:
        aim += x[2]
    # print(forward, depth, aim)
p2 = forward * depth
print('Part 2: ', p2)

Part 2:  1975421260


# --- Day 3: Binary Diagnostic ---

```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```

In [8]:
d = np.genfromtxt('data/aoc2021_day3.txt', dtype=int, delimiter=1)
gamma = epsilon = ''
for x in range(0,np.shape(d)[1]):
    c = Counter(d[:,x])
    gamma += str(c.most_common(1)[0][0]) #Most common
    epsilon += str(c.most_common(2)[1][0]) #Least common
p1 = int(gamma,2) * int(epsilon,2)
print('Part 1: ', p1)

Part 1:  1131506


In [9]:
def bit_count(col):
    """Find the most common bit value in an array (either 0 or 1)"""
    if np.sum(col) / len(col) >= 0.5:
        return 1
    else:
        return 0
    
def new_rows(rows, col_idx, bit_criteria):
    """
    Find new rows that match the bit criteria
    
    :param rows: two dimensional array
    :param col_idx: which column of the array to consider
    :param bit_criteria: 'most' for O2 rating, 'least' for CO2 scrubber rating
    :return: a new two dimensional array
    """
    keep_row = []
    most_common_bit = bit_count(rows[:,col_idx])
    if bit_criteria == 'most':
        bit_value = most_common_bit
    elif bit_criteria == 'least':
        bit_value = abs(1-most_common_bit) # flip 1 to 0, and 0 to 1
    for row in rows:
        if row[col_idx]==bit_value:
            keep_row.append(list(row))
    return np.array(keep_row)

def array_of_bits_to_decimal(arr):
    new_string = ''
    for x in arr:
        new_string += str(x)
    return int(new_string, 2)

In [10]:
nrows, ncols = np.shape(d)

rows = d
for col in range(0, ncols):
    rows = new_rows(rows, col, 'most')
    if len(rows) == 1:
        break
O2_gen_rating = array_of_bits_to_decimal(rows[0])

rows = d
for col in range(0, ncols):
    rows = new_rows(rows, col, 'least')
    if len(rows) == 1:
        break
CO2_scrubber_rating = array_of_bits_to_decimal(rows[0])

p2 = O2_gen_rating * CO2_scrubber_rating
print('Part 2: ', p2)

Part 2:  7863147


# --- Day 4: Giant Squid ---

In [11]:
puzzle_input = 'data/aoc2021_day4.txt'
draw_numbers = np.loadtxt(puzzle_input, dtype='int', delimiter = ',',max_rows=1)
draw_numbers

array([59, 91, 13, 82,  8, 32, 74, 96, 55, 51, 19, 47, 46, 44,  5, 21, 95,
       71, 48, 60, 68, 81, 80, 14, 23, 28, 26, 78, 12, 22, 49,  1, 83, 88,
       39, 53, 84, 37, 93, 24, 42,  7, 56, 20, 92, 90, 25, 36, 34, 52, 27,
       50, 85, 75, 89, 63, 33,  4, 66, 17, 98, 57,  3,  9, 54,  0, 94, 29,
       79, 61, 45, 86, 16, 30, 77, 76,  6, 38, 70, 62, 72, 43, 69, 35, 18,
       97, 73, 41, 40, 64, 67, 31, 58, 11, 15, 87, 65,  2, 10, 99])

In [12]:
# Read into separate boards
boards = []
with open(puzzle_input) as f:
    next(f) # Skip the first row of draw_numbers
    for line in f:
        if line != '\n':
            row = [int(s) for s in line.split()]
            boards.append(row)
boards=np.array(boards)
nboards = int(np.shape(boards)[0]/5)
boards = np.split(boards, nboards)
boards[0]

array([[42, 47, 77, 49, 67],
       [64, 82, 32, 94, 78],
       [96, 62, 45, 11, 43],
       [55, 92, 81, 66, 88],
       [12, 95, 19, 24, 71]])

In [13]:
def check_bingo(draw_board): # draw_board = draws[0], draws[1] etc.
    """Return 1 if a full row or full column equals 1"""
    nrow, ncol = np.shape(draw_board)
    for row in range(0,nrow):
        if np.sum(draw_board[row])==nrow:
            return 1
    for col in range(0,ncol):
        if np.sum(draw_board[:, col])==ncol:
            return 1
    return 0

In [14]:
# Initialize a bingo array
draws = np.zeros(np.shape(boards))
break_outer_loop = False
for number in draw_numbers:
    if break_outer_loop:
        break
    for x in range(0, len(boards)):
        draws[x][boards[x] == number] = 1
        if check_bingo(draws[x]):
            print('Bingo!')
            final_number = number
            board_number = x
            break_outer_loop = True
            break
print(board_number)
print(final_number)

Bingo!
41
60


In [15]:
sum_unmarked=sum(boards[board_number][draws[board_number]==0])
p1 = sum_unmarked * final_number
print('Part 1: ', p1)

Part 1:  46920


In [16]:
# Part 2, find the last board to win
finished_boards = set()
draws = np.zeros(np.shape(boards))
break_outer_loop = False
for number in draw_numbers:
    if break_outer_loop:
        break
    for x in range(0, len(boards)):
        draws[x][boards[x] == number] = 1
        if check_bingo(draws[x]):  
            finished_boards.add(x)
            if len(finished_boards) == nboards:
                print('Last board!')
                break_outer_loop = True
                final_number = number
                board_number = x
                break
print(board_number)
print(final_number)

Last board!
29
35


In [17]:
sum_unmarked=sum(boards[board_number][draws[board_number]==0])
p2 = sum_unmarked * final_number
print('Part 2: ', p2)

Part 2:  12635


# --- Day 5: Hydrothermal Venture ---
```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
```

In [18]:
#puzzle_input = 'data/test.txt'
puzzle_input = 'data/aoc2021_day5.txt'
lines = []
with open(puzzle_input) as f:
    for line in f:
        vent_lines = line.replace(' -> ', ',').strip().split(',')
        lines.append([int(x) for x in vent_lines])
lines[:10]

[[72, 504, 422, 154],
 [877, 851, 680, 654],
 [447, 989, 517, 989],
 [173, 125, 981, 933],
 [736, 255, 374, 617],
 [835, 681, 693, 539],
 [451, 176, 451, 885],
 [793, 629, 793, 157],
 [907, 945, 47, 85],
 [868, 104, 892, 104]]

In [19]:
def points_covered(p1, p2):
    """Given a line segment from p1 to p2, return all points covered"""
    # Ugly function, but it works..
    row = p2[0]-p1[0]
    col = p2[1]-p1[1]
    angle = math.degrees(math.atan2(p2[0]-p1[0],p2[1]-p1[1]))
    if angle == 0 or abs(angle) == 90 or abs(angle) == 180:
        pts = []
        if row == 0:
            if col>0:
                for x in range(p1[1], p2[1]+1):
                    pts.append([p1[0], x])
            else:
                for x in range(p2[1], p1[1]+1):
                    pts.append([p1[0], x])
        elif col == 0:
            if row > 0:
                for x in range(p1[0], p2[0]+1):
                    pts.append([x, p1[1]])
            else:
                for x in range(p2[0], p1[0]+1):
                    pts.append([x, p1[1]])
        return pts
                
    if angle == -45:
        t1 = list(range(p2[0]+1, p1[0]+1))
        t2 = list(reversed(range(p1[1],p2[1])))
    elif angle == 135:
        t1 = list(range(p1[0], p2[0]))
        t2 = list(reversed(range(p2[1]+1,p1[1]+1)))
    elif angle == 45:
        t1 = list(range(p1[0], p2[0]))
        t2 = list(range(p1[1],p2[1]))
    elif angle==-135:
        t1 = list(range(p1[0], p2[0], -1))
        t2 = list(range(p1[1],p2[1], -1))

    pts=[p2]
    for x in zip(t1,t2):
        pts.append(x)
    return pts

def fill_diagram(diagram, pts):
    """Fill a diagram with points"""
    for p in pts:
        diagram[p[0], p[1]] += 1
    return diagram

In [20]:
# PART 1 -------------------

# Initialise a diagram to count overlapping lines
diagram = np.zeros([np.max(lines)+1,np.max(lines)+1])

straight_lines = []
for line in lines:
    if line[0]==line[2] or line[1]==line[3]:
        straight_lines.append(line)

for line in straight_lines:
    p1 = line[0:2]
    p2 = line[2:]
    pts = points_covered(p1,p2)
    diagram = fill_diagram(diagram, pts)
print(diagram)
print('Part 1: ', np.sum(diagram>1))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Part 1:  6666


In [21]:
# PART 2 -------------------
# This time include both straight lines and angled lines
diagram = np.zeros([np.max(lines)+1,np.max(lines)+1])
for line in lines:
    p1 = line[0:2]
    p2 = line[2:]
    pts = points_covered(p1,p2)
    diagram = fill_diagram(diagram, pts)
print('Part 2: ', np.sum(diagram>1))

Part 2:  19081


In [22]:
# DEBUG (use this on the test data to find errors)
for line in lines:
    p1 = line[0:2]
    p2 = line[2:]
    angle = math.degrees(math.atan2(p2[0]-p1[0],p2[1]-p1[1]))
    #print(angle)
    pts = points_covered(p1,p2)
    #print('Line: {}'.format(line))
    #print(pts)

# --- Day 6: Lanternfish ---
```
Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
```

In [23]:
#puzzle_input = 'data/aoc2021_day7_test.txt'
puzzle_input = 'data/aoc2021_day6.txt'
fish = list(np.loadtxt(puzzle_input, dtype='int', delimiter = ','))
fish[0:10]

[1, 1, 3, 5, 1, 3, 2, 1, 5, 3]

In [24]:
# This function works fine for Part 1 (80 days), 
# but uses too much memory for Part 2 (256 days)
def spawn():
    a = fish
    while True:
        a, b = [6 if x == 0 else x-1 for x in a], [8 for x in a if x == 0]
        yield a+b
        a = a+b  # Add new fishes (b) to existing fishes (a)

#list(itertools.islice(spawn(), 34)) # print example output
p1 = len(list(itertools.islice(spawn(), 80))[-1])
print('Part 1: ', p1)

Part 1:  360610


In [25]:
# For Part 2, modify the spawn function and use an alternative approach
def spawn(i):
    """Find the number of fishes created by a 'fish number' i=1,2,3,4,5"""
    a = [i]
    day = 0
    while True:
        day += 1
        a, b = [6 if x == 0 else x-1 for x in a], [8 for x in a if x == 0]
        yield day, sum(1 for dummy in a+b), a+b
        a = a+b  # Add new fishes (b) to existing fishes (a)

# Example, for a fish with 3 days until spawning, this fish will have created
# 7 new fish after 20 days (one of these will have the number 1, one will have the
# number 4, three will have the number 6 and two will have the number 8)
for day, n_fishes, list_fishes in spawn(3):
        if day == 20:
            break
day, n_fishes, list_fishes

(20, 7, [4, 6, 6, 1, 6, 8, 8])

The above function becomes slow for day ~ 220. Hence it is first calculated how many fishes there will be after 156 days, and then it is calculated how many fishes these will produce after 100 days. In total it is thus calculated how many fishes there will be after 256 days.

In [26]:
# For the 5 'fish numbers', find how many fish each of these will create after X days
fish_count = {}
for f in [1,2,3,4,5]:
    for day, n_fishes, list_fishes in spawn(f):
        if day == 156:
            break
    counter=collections.Counter(list_fishes)
    fish_count[f]=dict(counter)
fish_count

{1: {6: 127909,
  1: 126808,
  3: 168344,
  5: 186314,
  0: 81567,
  2: 70737,
  4: 81396,
  7: 93709,
  8: 80581},
 2: {0: 80581,
  2: 126808,
  4: 168344,
  6: 186314,
  1: 81567,
  3: 70737,
  5: 81396,
  7: 47328,
  8: 93709},
 3: {1: 80581,
  3: 126808,
  5: 168344,
  0: 93709,
  2: 81567,
  4: 70737,
  6: 81396,
  7: 92605,
  8: 47328},
 4: {2: 80581,
  4: 126808,
  6: 168344,
  1: 93709,
  3: 81567,
  5: 70737,
  0: 47328,
  7: 34068,
  8: 92605},
 5: {3: 80581,
  5: 126808,
  0: 92605,
  2: 93709,
  4: 81567,
  6: 70737,
  1: 47328,
  7: 75739,
  8: 34068}}

In [27]:
# Calculate how many fish that will be created per fish number for the remaining days
list_last_fishes = []
for f in [0,1,2,3,4,5,6,7,8]:
    for day, n_fishes, list_fishes in spawn(f):
        #print(day, n_fishes, list_fishes)
        if day == 100:
            break
    list_last_fishes.append(n_fishes)
list_last_fishes # Element 0 is for fish number 0, element 1 is for fish number 1 etc.

[8492, 7776, 6983, 6686, 5762, 5629, 4837, 4659, 4164]

In [28]:
# Calculate the total number of fishes
def calc_sum(fish_count, list_last_fishes):
    sum_fishes = 0
    for i in range(0,8+1):
        try:
            sum_fishes += fish_count[i]*list_last_fishes[i]
        except KeyError:
            continue
    return sum_fishes

sum_per_number = {}
for k, v in fish_count.items():
    sum_per_number[k]=calc_sum(v, list_last_fishes)
sum_per_number

{1: 6206821033, 2: 5617089148, 3: 5217223242, 4: 4726100874, 5: 4368232009}

In [29]:
# Find the total number of fishes for all the input numbers
input_count=dict(collections.Counter(fish))
print(input_count)
total_sum = 0
for i in [1,2,3,4,5]:
        try:
            total_sum += input_count[i]*sum_per_number[i]
        except KeyError:
            continue
print('Part 2: ', total_sum)

{1: 117, 3: 41, 5: 50, 2: 43, 4: 49}
Part 2:  1631629590423


# --- Day 7: The Treachery of Whales ---
```
16,1,2,0,4,2,7,1,2,14
```

In [30]:
#puzzle_input = 'data/test.txt'
puzzle_input = 'data/aoc2021_day7.txt'
pos = np.loadtxt(puzzle_input, dtype='int', delimiter = ',')
pos[0:10]

array([1101,    1,   29,   67, 1102,    0,    1,   65, 1008,   65])

In [31]:
# Part 1
min_cost = 999999999999999999
for x in range(min(pos), max(pos)+1):
    cost = sum(np.abs(pos-x))
    if cost < min_cost:
        min_cost = cost
print('Part 1: ', min_cost)

Part 1:  341558


In [32]:
def firstn(n):
    num = 0
    while num < n:
        num += 1
        yield num
        
sum(firstn(9)) # 1+2+3+4+5+6+7+8+9

45

In [33]:
# Part 2
min_cost = 999999999999999999
for x in range(0, max(pos)+1):
    delta = np.abs(pos-x)
    new_cost = 0
    for y in delta:
        new_cost+=sum(firstn(y))
    if new_cost < min_cost:
        min_cost = new_cost
print('Part 2: ', min_cost)

Part 2:  93214037
